# 🐼 Pandas ServiceNow Session — End-to-End Demo
This notebook covers **read → wrangle → analyze → visualize → export** using a ServiceNow-style dataset.

## 📦 Imports + Quick Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read CSVs (adjust paths if needed)
tickets = pd.read_csv("servicenow_demo_tickets.csv", parse_dates=["opened_at","closed_at"])
users   = pd.read_csv("servicenow_demo_users.csv")

tickets.head()

> **Note:** Think of `tickets` as your **Incident** table, and `users` as a lookup for assignee metadata.

## 🔗 1) Join vs Merge — What's the difference?

- **`merge`**: SQL-style join on columns (explicit & most common)
- **`join`**: Convenience join on **index** (or a column via `on=`), handy after you set an index

In [ ]:
# Merge (left join)
enriched = tickets.merge(
    users[["assignee","assignee_full_name","location","manager"]],
    on="assignee", how="left", validate="m:1"
)
enriched.head()

**Why:** Add human-friendly assignee details to tickets.

**Tip:** `validate="m:1"` guards against accidental many-to-many explosions.

In [ ]:
# Join (after setting index)
enriched2 = (
    tickets.set_index("assignee")
           .join(users.set_index("assignee"), how="left")
           .reset_index()
)
enriched2.head()

**Why:** A bit shorter once keys are indices; functionally similar to `merge`.

## 🗑️ 2) Deleting rows with null values

In [ ]:
# Drop rows where assignee OR priority is missing
clean_rows = tickets.dropna(subset=["assignee", "priority"])
clean_rows.head()

**Why:** Enforce critical fields before KPIs.

**Analogy:** Reject incomplete forms before processing.

## 🗑️ 3) Deleting columns with (mostly) null values

In [ ]:
# Drop columns that are 80% or more missing
threshold = 0.8
cols_to_drop = tickets.columns[tickets.isna().mean() >= threshold]
tickets_dropcols = tickets.drop(columns=cols_to_drop)
tickets_dropcols.head()

**Why:** Remove noisy columns that won’t help analysis.

## 📑 4) Read Excel (specific sheet/tab)

In [ ]:
# Read "Tickets" sheet
tickets_xlsx = pd.read_excel("servicenow_demo.xlsx", sheet_name="Tickets", parse_dates=["opened_at","closed_at"])

# Read "Users" sheet
users_xlsx = pd.read_excel("servicenow_demo.xlsx", sheet_name="Users")

**Why:** Enterprise data often lands as Excel with multiple tabs.

## 🌐 5) Read JSON and Parquet

In [ ]:
# JSON (records)
tickets_json = pd.read_json("servicenow_demo_tickets.json")

# Parquet (fast, typed)
tickets_parquet = pd.read_parquet("servicenow_demo_tickets.parquet")

**Why:** JSON is common for APIs; Parquet is best for speed + schema.

## 🧹 6) Imputing missing values (fillna, per-type strategies)

In [ ]:
# 6.1 Simple fills
tickets["priority"] = tickets["priority"].fillna("P3")  # fill categorical with a default
tickets["assignment_group"] = tickets["assignment_group"].fillna("Unassigned")
tickets["time_to_resolve_hours"] = tickets["time_to_resolve_hours"].fillna(0.0)

# 6.2 Group-wise imputation (e.g., fill priority by category mode)
mode_by_cat = (tickets.groupby("category")["priority"]
                      .agg(lambda s: s.mode().iloc[0] if not s.mode().empty else "P3"))
tickets["priority"] = tickets["priority"].fillna(tickets["category"].map(mode_by_cat))

**Why:** Missing-value policy should follow business rules (defaults, mode/median, or group-wise fills).

## 🔍 7) Selecting and filtering

In [ ]:
# Select a few columns
core = tickets[["ticket_id","opened_at","state","priority","assignment_group","assignee","customer"]]

# Filter high-priority open tickets
mask = (tickets["priority"].isin(["P1","P2"])) & (~tickets["state"].isin(["Resolved","Closed"]))
open_high = tickets.loc[mask, ["ticket_id","priority","state","assignment_group","assignee"]]
open_high.head()

**Analogy:** Think slicers/filters in Excel—only scripted and repeatable.

## 📊 8) Grouping and aggregating (KPIs)

In [ ]:
# Average TTR and count by assignment group
kpi_group = (tickets
             .groupby("assignment_group", as_index=False)
             .agg(
                 tickets_count=("ticket_id","count"),
                 avg_ttr_hours=("time_to_resolve_hours","mean"),
                 p90_ttr=("time_to_resolve_hours", lambda s: s.quantile(0.9))
             )
             .sort_values("tickets_count", ascending=False))
kpi_group.head()

In [ ]:
# Monthly volume by priority
tickets["month"] = tickets["opened_at"].dt.to_period("M").astype(str)
vol_by_month_priority = (tickets
    .groupby(["month","priority"], as_index=False)
    .agg(count=("ticket_id","count"))
    .sort_values(["month","priority"]))
vol_by_month_priority.head()

**Why:** These rollups become dashboard tiles and SLO inputs.

## 🗂️ 9) Sort and deduplicate (keep latest per ticket or per key)

In [ ]:
# Keep the latest record per ticket_id by closed_at (or opened_at if still open)
tickets_sorted = tickets.sort_values(["ticket_id","closed_at","opened_at"], ascending=[True, False, False])
latest_per_ticket = tickets_sorted.drop_duplicates(subset="ticket_id", keep="first")
latest_per_ticket.head()

**Why:** Real tables often have multiple snapshots; pick the canonical one.

## 🩹 10) Handling missing values (overview)

In [ ]:
# Missingness profile (top 5)
missing_report = tickets.isna().mean().sort_values(ascending=False).head(5)
missing_report

In [ ]:
# Replace booleans and flags
tickets["sla_breach"] = tickets["sla_breach"].fillna(False)

# Drop rows missing BOTH assignee and assignment_group
drop_both = tickets.dropna(subset=["assignee", "assignment_group"], how="all")
drop_both.head()

**Why:** Know where holes are; decide: fill, drop, or model.

## 📈 11) Graphs using matplotlib

Run each cell separately to generate one plot at a time.

In [ ]:
# Bar chart: Tickets by priority
count_by_priority = tickets["priority"].value_counts().sort_index()
plt.figure()
count_by_priority.plot(kind="bar")
plt.title("Ticket Count by Priority")
plt.xlabel("Priority")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

In [ ]:
# Line chart: Monthly volume
vol_month = tickets.groupby("month")["ticket_id"].count().sort_index()
plt.figure()
vol_month.plot(kind="line", marker="o")
plt.title("Monthly Ticket Volume")
plt.xlabel("Month")
plt.ylabel("Tickets")
plt.tight_layout()
plt.show()

In [ ]:
# Histogram: Time to resolve (hours)
plt.figure()
tickets["time_to_resolve_hours"].dropna().plot(kind="hist", bins=15)
plt.title("Distribution of Time to Resolve (Hours)")
plt.xlabel("Hours")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

## 🔗 Extra: a clean, readable pipeline (method chaining)

In [ ]:
# From raw tickets → enrich with users → KPI rollup
kpi = (
    pd.read_csv("servicenow_demo_tickets.csv", parse_dates=["opened_at","closed_at"])
      .assign(
          month=lambda d: d["opened_at"].dt.to_period("M").astype(str),
          time_to_resolve_hours=lambda d: d["time_to_resolve_hours"].fillna(0.0),
          priority=lambda d: d["priority"].fillna("P3"),
      )
      .merge(pd.read_csv("servicenow_demo_users.csv")[['assignee','location']], on="assignee", how="left")
      .groupby(["month","priority","location"], as_index=False)
      .agg(tickets=("ticket_id","count"),
           avg_ttr=("time_to_resolve_hours","mean"))
      .sort_values(["month","priority","location"])
)
kpi.head()

**Why:** Reads like a recipe you can re-run every day.

## 💾 12) Writing / Exporting Data

In [ ]:
# To CSV
tickets.to_csv("tickets_clean.csv", index=False)

# To Excel with multiple sheets
with pd.ExcelWriter("tickets_output.xlsx", engine="xlsxwriter") as writer:
    tickets.to_excel(writer, sheet_name="Tickets", index=False)
    users.to_excel(writer, sheet_name="Users", index=False)

# To JSON
tickets.to_json("tickets.json", orient="records", indent=2, date_format="iso")

# To Parquet
tickets.to_parquet("tickets.parquet", index=False)

- `index=False` prevents row numbers from being written as a column
- `orient="records"` makes row-wise JSON objects; `date_format="iso"` keeps ISO timestamps
- Parquet is fast/typed and ideal for big-data pipelines (Spark/Dask/Athena)

## ✅ Tips for your session- Call out **why** each step matters (e.g., imputation tied to SLA reporting)
- Encourage participants to tweak thresholds (e.g., P1/P2 filter) and re-run
- Show a merge with `validate=` to demonstrate safe joins
- Close by saving outputs with `to_csv` / `to_parquet`